In [11]:
import openai
import base64
import os
from PIL import Image
from io import BytesIO  
import mykey
openai_client = openai.OpenAI(api_key = "mykey.OPENAI_KEY")
using_model = 'gpt-4o-mini'

# 53개 이미지에 0.02$ , 30원 ! 이미지당 1원 안댐~!

In [9]:
# Function to encode the image
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")
    
# 이미지 영역을 잘라서 Base64로 인코딩하는 함수
def encode_cropped_image(image_path, coordinates):
    try:
        # 이미지 열기
        with Image.open(image_path) as img:
            # RGBA 모드일 경우 RGB로 변환
            if img.mode == "RGBA":
                img = img.convert("RGB")
            
            # 좌표 가져오기
            xtl, ytl, xbr, ybr = map(float, [
                coordinates["xtl"],
                coordinates["ytl"],
                coordinates["xbr"],
                coordinates["ybr"]
            ])
            # 이미지 자르기
            cropped_img = img.crop((xtl, ytl, xbr, ybr))
            
            # 이미지를 메모리에 저장 후 Base64로 인코딩
            buffer = BytesIO()
            cropped_img.save(buffer, format="JPEG")
            return base64.b64encode(buffer.getvalue()).decode("utf-8")
    except Exception as e:
        print(f"Error processing {image_path} with coordinates {coordinates}: {e}")
        return None


In [10]:
import xml.etree.ElementTree as ET

# XML 파일 경로
xml_file = "/home/bongo/porter_notebook/yolo/aihub/data_test/yolo_test/origin_label_xml/Suwon_CH01_20200720_1730_MON_9m_NH_highway_TW5_sunny_FHD.xml"
adjust_xml_file = "/home/bongo/porter_notebook/yolo/aihub/data_test/yolo_test/adjust_label_xml/Suwon_CH01_20200720_1730_MON_9m_NH_highway_TW5_sunny_FHD.xml"


# XML 파싱
tree = ET.parse(xml_file)
root = tree.getroot()
     
image_folder = "/home/bongo/porter_notebook/yolo/aihub/data_test/yolo_test/origin_img"
# car 레이블 추출
car_labels = []
for image in root.findall("image"):
    image_name = image.attrib.get("name")
    image_path = f"{image_folder}/{image_name}"
    
    if not os.path.exists(image_path):
        continue
    encoded_image = encode_image(image_path)  # 이미지 인코딩
    print(f"image_path : {image_path}")
    for box in image.findall("box"):
        if box.attrib.get("label") == "car":
            coordinates = {
                "xtl": box.attrib.get("xtl"),
                "ytl": box.attrib.get("ytl"),
                "xbr": box.attrib.get("xbr"),
                "ybr": box.attrib.get("ybr"),
            }
            # Boundary 영역만 Base64 인코딩
            encoded_image = encode_cropped_image(image_path, coordinates)
            
            car = {
                "image_name": image_name,
                "coordinates": {
                    "xtl": box.attrib.get("xtl"),
                    "ytl": box.attrib.get("ytl"),
                    "xbr": box.attrib.get("xbr"),
                    "ybr": box.attrib.get("ybr"),
                },
                "z_order": box.attrib.get("z_order"),
                "encoded_image": encoded_image,  # Base64 인코딩된 이미지
            }
            car_labels.append(car)
            messages = [
            {
                "type": "text",
                "text": '이미지를 바탕으로 차량이 SUV/sedan/VAN/compact_car 중 무엇인지 구분해서 SUV/sedan/VAN/compact_car/etc 중 하나의 단어로만 '
            },
            {
                "type": "image_url",
                "image_url": {"url": f"data:image/jpeg;base64,{car['encoded_image']}"}
            }
            ]
            # OpenAI API 요청
            response = openai_client.chat.completions.create(
                model="gpt-4o-mini",
                messages=[
                    {
                        "role": "user",
                        "content": messages
                    }
                ],
                max_tokens=300
            )
            # 결과 출력
            img_analy = response.choices[0].message.content
            print(f"result : {img_analy}")            
            # XML 내 label 업데이트
            box.set("label", img_analy)    
            # 수정된 XML 저장
            tree.write(adjust_xml_file, encoding="utf-8", xml_declaration=True)


image_path : /home/bongo/porter_notebook/yolo/aihub/data_test/yolo_test/origin_img/Suwon_CH01_20200720_1730_MON_9m_NH_highway_TW5_sunny_FHD_002.png
result : VAN
result : sedan
result : VAN
result : VAN
result : SUV
result : SUV
result : SUV
result : SUV
result : sedan
image_path : /home/bongo/porter_notebook/yolo/aihub/data_test/yolo_test/origin_img/Suwon_CH01_20200720_1730_MON_9m_NH_highway_TW5_sunny_FHD_003.png
result : SUV
result : compact_car
result : sedan
result : sedan
result : SUV
result : SUV
result : SUV
result : SUV
result : SUV
result : compact_car
result : SUV
result : SUV
result : sedan
result : SUV
result : SUV
result : VAN
result : sedan
result : SUV
result : SUV
image_path : /home/bongo/porter_notebook/yolo/aihub/data_test/yolo_test/origin_img/Suwon_CH01_20200720_1730_MON_9m_NH_highway_TW5_sunny_FHD_006.png
result : SUV
result : sedan
result : SUV
result : sedan
result : SUV
result : SUV
result : sedan
result : SUV
result : sedan
result : SUV
result : SUV
result : SUV


In [22]:
len(car_labels)

74

In [6]:
messages = [
    {
        "type": "text",
        "text": '이미지를 바탕으로 차량이 SUV/세단/준형 중 무엇인지 구분해서 SUV/세단/준형/기타 중 하나의 단어로만 '
    }
]

In [7]:
# car_labels 데이터를 메시지에 추가
for car in car_labels:
    messages.append({
        "type": "text",
        "text": f"Image: {car['image_name']}, Coordinates: {car['coordinates']}, Z-Order: {car['z_order']}"
    })
    messages.append({
        "type": "image_url",
        "image_url": {"url": f"data:image/jpeg;base64,{car['encoded_image']}"}
    })
    break


In [21]:
# OpenAI API 요청
response = openai.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "user",
            "content": messages
        }
    ],
    max_tokens=300
)

# 결과 출력
img_analy = response.choices[0].message.content
print(img_analy)

SUV


In [ ]:
import xml.etree.ElementTree as ET

# XML 파일 경로
xml_file = "/home/bongo/porter_notebook/yolo/aihub/data_test/yolo_test/origin_label_xml/Suwon_CH01_20200720_1730_MON_9m_NH_highway_TW5_sunny_FHD.xml"

# XML 파일 로드 및 파싱
tree = ET.parse(xml_file)
root = tree.getroot()

# 순서에 맞는 'car' 레이블 대체
new_labels = img_analy.split("\n")  # img_analy의 값을 라인별로 분리한다고 가정
car_index = 0  # 'car' 레이블 순서 추적

for image in root.findall("image"):
    for box in image.findall("box"):
        if box.attrib.get("label") == "car":
            if car_index < len(new_labels):
                # 기존 'car'를 새로운 레이블로 대체
                box.set("label", new_labels[car_index].strip())
                car_index += 1

# 수정된 XML 파일 저장
updated_xml_file = xml_file.replace(".xml", "_updated.xml")
tree.write(updated_xml_file, encoding="utf-8", xml_declaration=True)

print(f"Updated XML saved to {updated_xml_file}")
